<a href="https://colab.research.google.com/github/nickprock/appunti_data_science/blob/master/semantic-search/FAQ_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FAQ Search

<br>

![faq](https://cdn.mageplaza.com/docs/faq-kb-advanced-search-faq.gif)

<br>

In un momento storico in cui molte delle risposte che cerchiamo vengono **generate** dall'intelligenza artificiale (i noti modelli GPT) ci sono dei campi in cui fare affidamento solo a queste tecnologie non è del tutto consigliato, il primo che viene in mente è il campo medico dove il parere dell'esperto sia per ricadute sulla salute che per tematiche legali è ancora fondamentale.

Negli anni si è provato ad automatizzare questo processo, uno dei risultati è il proliferare di milioni di siti internet a sfondo medico che hanno contribuito a creare una nuova malattia, la ***cybercondria***.

Le FAQ hanno il vantaggio di non inventare nulla (se sono "certificate") ma allo stesso tempo in molti casi sono estremamente verticali su un argomento utilizzando un linguaggio molto specifico. Questo può essere un problema per i LLM che generano gli embeddings in quanto sono addestrati su testi generalisti e perdono i tecnicismi.

> Ho parlato ci cybercondria al PyCon22 [trovate il video qui](https://www.youtube.com/watch?v=zZyfQ4Pc-ek)

## Come procedere?

Si possono perseguire due strade:
* addestrare un modello sul mercato verticale, richiede tempo, risorse e soprattutto una fonte dati corposa da cui attingere.
* utilizzare la [hybrid search](https://weaviate.io/blog/hybrid-search-explained), ovvero combinare un algoritmo di sparse search come [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) ed un pretrained sentence transformer. Anche qui ci sono dei contro uno verrà citato in seguito.

Non esiste una strategia giusta da utilizzare, in questo notebook mostrerà la seconda, o almeno un modo di approcciare all'hybrid search, andando avanti cercherò di elencare le possibili variazioni sul tema.

<br>

![hs](https://weaviate.io/assets/images/hybrid-search-explained-6c9a3c81beb57e3c9c15cb5d996f249b.png)

### Haystack

Per creare il document store e il nostro motore di ricerca utilizzerò [Haystack](https://haystack.deepset.ai/) una liberia specializzata in ricerca semantica e Q&A che permette di costruire in maniera modulare search engine utilizzando vettori sia sparsi che densi.

### Dataset

Utilizzerò un piccolo dataset, circa 200 FAQ copiato dal sito dell'[AIRC](https://www.airc.it/).

### Pipeline

La FAQ pipeline sarà composta da:

* Document Store su cui verranno caricate le FAQ
* BM25 Engine
* Dense engine creato nel [notebook sul fine tuning dei Bi-Encoder](https://github.com/nickprock/appunti_data_science/blob/master/semantic-search/fine-tuning-sentence-transformer.ipynb)
* Concatenazione dei risultati
* Re-Rank mediante [cross-encoder](https://github.com/nickprock/appunti_data_science/blob/master/semantic-search/sentence-transformer-cross-encoder.ipynb)

> *N.B. il notebook è stato creato su Colab, se volete usare altri environment per l'installazione vi consiglio la documentazione ufficiale di haystack.*



In [1]:
%%bash

pip install --upgrade pip
pip install transformers
pip install farm-haystack[inference]

apt install libgraphviz-dev
pip install pygraphviz

Reading package lists...
Building dependency tree...
Reading state information...
libgraphviz-dev is already the newest version (2.42.2-3build2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


## Hybrid Search Pipeline

### DocumentStore

Un Haystack DocumentStore è un database che può memorizzare sia vettori che metadati, questi vengono passati al retriever quando arriva una query.

Haystack fornisce connettori per diversi VectorDB e SearchEngine come (ElasticSearch, Pinecone, Qdrant, ...) ma nel nostro caso avendo pochissimi dati usememo InMemoryDocumentStore.

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


### Retriever

Come elencato in precedenza ci servono due retriever:
* Sparse con BM25
* Dense con Sentence Transformer

Li inizializziamo semplicemente importando le classi dai nodi di haystack.

> Come Sentence Transformer ho scelto quello creato in [questo notebook](https://github.com/nickprock/appunti_data_science/blob/master/semantic-search/fine-tuning-sentence-transformer.ipynb)

In [4]:

from haystack.nodes import EmbeddingRetriever, BM25Retriever

sparse_retriever = BM25Retriever(document_store=document_store)

dense_retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="nickprock/sentence-bert-base-italian-uncased",
    batch_size=16,
    use_gpu=True,
    scale_score=False,
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model nickprock/sentence-bert-base-italian-uncased
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Caricare i dati sul DocumentStore

Come già accennato abbiamo un *.csv* di FAQ copiate dal sito dell'AIRC. I dati sono molto pochi ma rendono bene su questo task.
> Non ci aspettiamo grandissimi risultati in termini di vastità di query che si possono sottoporre.

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/airc.csv', sep=";")
# Minimal cleaning
df.fillna(value="", inplace=True)
df["Domanda"] = df["Domanda"].apply(lambda x: x.strip())
df.head()

,Domanda,Risposta
0,quanto incidono le abitudini alimentari sul ri...,l'american institute for cancer research ha ca...
1,quali sostanze possono favorire lo sviluppo de...,i nitriti e i nitrati utilizzati per la conser...
2,esistono tumori legati più di altri al tipo di...,la risposta è sì: ci sono tumori più sensibili...
3,quali sono i tumori che risentono maggiormente...,"i tumori dell'apparato gastrointestinale, e in..."
4,cosa mangiare per cercare di prevenire la mala...,occorre portare a tavola almeno cinque porzion...


A questo punto bisogna creare gli embeddings delle domande, in questo caso particolare non ci serve l'embedding di entrambe le parti perchè la ricerca viene fatta solo sulla domanda e le risposte vengono solo proposte senza nessuna elaborazione.

Al termine carichiamo tutto sul DocumentStore.

In [7]:
df = df.rename(columns={"Domanda": "content", "Risposta":"answer"})

I documenti vengono caricati nel DocumentStore, il content è la parte che viene "tradotta" in vettore, il resto và nei metadati. Viene anche creato l'indice per il  BM25.

In [8]:
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

Updating BM25 representation...:   0%|          | 0/228 [00:00<?, ? docs/s]

Con la cella successiva verranno caricati gli embeddings nel document store.

In [9]:
document_store.update_embeddings(dense_retriever)

INFO:haystack.document_stores.memory:Updating embeddings for 0 docs ...


Updating Embedding:   0%|          | 0/228 [00:00<?, ? docs/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
document_store.get_all_documents(return_embedding=True)[:2]

[<Document: {'content': 'quanto incidono le abitudini alimentari sul rischio di sviluppare un tumore?', 'content_type': 'text', 'score': None, 'meta': {'answer': "l'american institute for cancer research ha calcolato che le cattive abitudini alimentari sono responsabili di circa tre tumori su dieci"}, 'id_hash_keys': ['content'], 'embedding': '<embedding of shape (768,)>', 'id': '1153732664aa9ca86386d34c998266a1'}>,
 <Document: {'content': 'quali sostanze possono favorire lo sviluppo della malattia?', 'content_type': 'text', 'score': None, 'meta': {'answer': "i nitriti e i nitrati utilizzati per la conservazione dei salumi, per esempio, facilitano la comparsa del tumore dello stomaco, più in generale gli studi epidemiologici hanno dimostrato che un'alimentazione ricca di grassi e proteine animali favorisce la comparsa della malattia"}, 'id_hash_keys': ['content'], 'embedding': '<embedding of shape (768,)>', 'id': 'd532c1e436204f60c995c9b5f8d5c61d'}>]

## Costruire la Pipeline

Haystack ha nella sua cassetta degli attrezzi moltissime pipipeline tra cui quella per le FAQ, però noi vogliamo un hybrid retriever e questo dobbiamo costruircelo.

* Per prima cosa verranno aggiunti alla pipeline due rami, i due retriever che prendono in input la query.
* I risultati dei due verranno riversati in un nodo di unione `JoinDocuments`, questo nodo calcola uno score, [può anche essere usato come punto rerank](https://docs.haystack.deepset.ai/docs/join_documents) (non è il nostro caso). Noi non vogliamo un suo score ma solo che concateni i documenti eliminando i duplicati.
* I documenti estratti verranno passati al reranker, ovvero il nostro cross encoder che ci dirà quali sono i documenti più simili a cio che abbiamo richiesto.

> Come per il Dense Retriever anche il Reranker è un [Cross-Encoder su cui avevo fatto Fine Tuning precedentemente](https://github.com/nickprock/appunti_data_science/blob/master/semantic-search/sentence-transformer-cross-encoder.ipynb).




In [11]:
from haystack.nodes import JoinDocuments, SentenceTransformersRanker

In [12]:
join_documents = JoinDocuments()
rerank = SentenceTransformersRanker("nickprock/cross-encoder-italian-bert-stsb")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Ora che abbiamo tutte le componenti è il momento di andare a costruire l'hybrid search engine un livello alla volta.

Farlo è molto semplice, inizializziamo una piepline di base e aggiungiamo le componenti, per ogni componente aggiunta và specificato:
* nome: stringa
* componente di base: l'oggetto che aggiungiamo
* gli input: lista di stringhe

In [13]:
from haystack.pipelines import Pipeline

hybrid_retriever = Pipeline()
hybrid_retriever.add_node(component = sparse_retriever,name = "sparseRetriever", inputs=["Query"])
hybrid_retriever.add_node(component = dense_retriever, name = "denseRetriever", inputs=["Query"])
hybrid_retriever.add_node(component = join_documents, name = "joinDocuments", inputs=["sparseRetriever", "denseRetriever"])
hybrid_retriever.add_node(component = rerank, name = "reranker", inputs=["joinDocuments"])


possiamo farci generare un *.png* della pipeline facilmente con il metodo `.draw()`

In [14]:
hybrid_retriever.draw()

<br>

![pipe](https://github.com/nickprock/appunti_data_science/blob/master/semantic-search/content/pipeline.png?raw=true)

<br>

Ora possiamo far eseguire la nostra query, inseriamo alcuni parametri presenti ai vari livelli dell'hybrid search:
* Dai due retriever ci facciamo restituire 10 documenti a testa, quindi nella peggiore delle ipotesi avremo 20 documenti da concatenare
* Il nodo di join ci restituisce i primi dieci
* questi dieci vengono passati al reranker che li confronta con la query e ci darà il più simile

In [15]:
prediction = hybrid_retriever.run(query="Cosa sono i marker tumorali?", params={"sparseRetriever":{"top_k": 10},
                                                                                "denseRetriever":{"top_k": 10},
                                                                                "joinDocuments":{"top_k_join": 10},
                                                                                "reranker": {"top_k": 1}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Nelle prossime due celle vediamo la risposta completa e se vogliamo estrarre solo la `answer`.

In [16]:
prediction

{'documents': [<Document: {'content': 'cosa sono i marcatori tumorali?', 'content_type': 'text', 'score': 0.848907470703125, 'meta': {'answer': 'i cosiddetti marcatori tumorali fanno parte degli strumenti per determinare il rischio tumorale: si tratta di molecole che, se presenti o assenti nel sangue, permettono di capire se una persona è o no a rischio di sviluppare un determinato tipo di tumore.'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '23f22165cdddab5334e67e340dda1798'}>],
 'labels': None,
 'root_node': 'Query',
 'params': {'sparseRetriever': {'top_k': 10},
  'denseRetriever': {'top_k': 10},
  'joinDocuments': {'top_k_join': 10},
  'reranker': {'top_k': 1}},
 'query': 'Cosa sono i marker tumorali?',
 'node_id': 'reranker'}

In [17]:
prediction['documents'][0].meta['answer']

'i cosiddetti marcatori tumorali fanno parte degli strumenti per determinare il rischio tumorale: si tratta di molecole che, se presenti o assenti nel sangue, permettono di capire se una persona è o no a rischio di sviluppare un determinato tipo di tumore.'

### Debug Pipeline

Per indagare al meglio sui layer della pipeline possiamo andare in debug. Una cosa che non ho approfondito sono gli score dei documenti, diversi retriever danno score diversi e non sempre equiparabili, questo è un problema aperto della hybrid search, se volete approfondire rimando ad un bel post dal blog di [Qdrant](https://qdrant.tech/articles/hybrid-search/) e ad uno da quello di [Pinecone](https://www.pinecone.io/learn/hybrid-search-intro/).

Per andare in debug basta aggiungere `"debug" = True` ai parametri del livello su cui volete indagare.

> N.B. Io l'ho messo a tutti, pessima scelta perchè dai log non si capisce nulla.

In [18]:
prediction_debug = hybrid_retriever.run(query="Cosa sono i marker tumorali?", params={"sparseRetriever":{"top_k": 10, "debug": True},
                                                                                "denseRetriever":{"top_k": 10, "debug": True},
                                                                                "joinDocuments":{"top_k_join": 10, "debug": True},
                                                                                "reranker": {"top_k": 1, "debug": True}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
prediction_debug

{'documents': [<Document: {'content': 'cosa sono i marcatori tumorali?', 'content_type': 'text', 'score': 0.848907470703125, 'meta': {'answer': 'i cosiddetti marcatori tumorali fanno parte degli strumenti per determinare il rischio tumorale: si tratta di molecole che, se presenti o assenti nel sangue, permettono di capire se una persona è o no a rischio di sviluppare un determinato tipo di tumore.'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '23f22165cdddab5334e67e340dda1798'}>],
 '_debug': {'sparseRetriever': {'input': {'root_node': 'Query',
    'query': 'Cosa sono i marker tumorali?',
    'top_k': 10,
    'debug': True},
   'output': {'documents': [<Document: {'content': 'cosa sono i marcatori tumorali?', 'content_type': 'text', 'score': 0.7794130015189505, 'meta': {'answer': 'i cosiddetti marcatori tumorali fanno parte degli strumenti per determinare il rischio tumorale: si tratta di molecole che, se presenti o assenti nel sangue, permettono di capire se una persona è o 

## Conclusioni

Il notebook è introduttivo e credo sia una base esaustiva e con qualche link di appronfondimento sul tema per chi cerca qualcosa di più avanzato.

Il consiglio che dò a chi si approccia a questa tematica per la prima volta è attivare e disattivare dei livelli nella pipeline e confrontare i risultati restituiti, forse il dataset è troppo piccolo ma la ricerca dovrebbe comunque trarre giovamento dal doppio retriever.

*Come sempre i vostri feedback sono ben accetti!*

**Buona Ricerca 😃**